In [1]:
# from shiny import App, render, ui, reactive
# from shinywidgets import output_widget, render_widget
# import seaborn as sns
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys

sys.path.insert(1, '/homes_unix/agillig/Projects/FabricOfCognition/code')
import func_toolbox as ftools
from func_toolbox import fetch_neurosynth_data

# import plotly.graph_objects as go
import numpy as np
import pandas as pd
from pathlib import Path
import os


from nilearn import image

project_dir = '/homes_unix/agillig/github_repos/ginna' #indicate path to the repository

neurosynth_terms_file = project_dir + '/data/terms/BCS_3D.csv'
os.makedirs(Path(neurosynth_terms_file).parent, exist_ok=True)

# download the file from the Pacela et al. 2021 paper repo 
# https://github.com/vale-pak/BCS

if not os.path.exists(neurosynth_terms_file):
    fetch_neurosynth_data(f'{project_dir}/data')

# https://github.com/vale-pak/BCS/blob/main/BCS_3D.csv
df = pd.read_csv(neurosynth_terms_file, sep = ',')

neurosynth_terms = df['Functions']

fcu = ftools.Utilities()
atlas_str = "aicha-aal3"

# create a new directory for the analysis
analysis_dir = project_dir + '/analysis/mean_RSNs_aicha-aal3'
os.makedirs(analysis_dir, exist_ok=True)

mip_rsn_dir = project_dir + f'/Results/mip/mip_rsn_parcellated/aicha'


# Data processing

## create atlas : aicha + AAL cerebellum / BG

In [2]:
parcellation_atlas_file = f'{project_dir}/data/parcellation_atlases/aicha-aal3/parcels_aicha-aal3.nii.gz'

# the atlas is already present in the github repository
if not os.path.exists(parcellation_atlas_file):
        atlases_dir = ''
        file_cortical = atlases_dir + '/AICHA_v2_websiteC/AICHA.nii'
        file_subcortical = atlases_dir + '/AAL3/AAL3v1.nii'

        cortical_img = image.load_img(file_cortical)
        new_data = np.zeros_like(cortical_img.get_fdata(), dtype = 'int32')

        cortical_data = image.load_img(file_cortical).get_fdata()
        subcortical_data = image.load_img(file_subcortical).get_fdata()
        # cortical
        max_cortical = np.max(cortical_data)
        for i, value in enumerate(np.unique(cortical_data)[1:]):
                new_data[cortical_data == value] = i + 1 
        # cerebellum
        # indices : from 95-120 for cerebellum; 121-170 for subcrotcial/brainstem
        for i, value in enumerate([l for l in range(95 , 121)]):
                new_data[subcortical_data == value] = max_cortical + i + 1 

        new_img = image.new_img_like(cortical_img, new_data)
        out_map = parcellation_atlas_file
        os.makedirs(os.path.dirname(out_map), exist_ok=True)
        # new_img.to_filename(out_map)


In [3]:
# proj = ftools.concatenate_all_rsn_projections()

# fcu_mean = FC.utilities()
# fcu_mean.analysis_dir = '/analysis/mean_RSNs'
# proj_m = fcu_mean.concatenate_all_rsn_projections()

# filestr = [f'rsn-{r:02d}'for r in df['rsn'].unique()]
# rsn_int = [int(i) for i in proj['rsn'].unique()]

# rsns to exclude
# exclude = ['05', '34', '40'] # 34 bg, 40 artifact

# rsn_int = [i for i in rsn_int if np.isin(f'{i:02d}', exclude) == False]

# rsn_str = [f'{el:02d}' for el in rsn_int]
# filestr = [f'{i:02d}' for i in rsn_int]

# n_rsn = len(rsn_str)

In [4]:
atlas_dir = f'{project_dir}/atlas/zmaps'
rsn_files = [os.path.join(atlas_dir, f) for f in os.listdir(atlas_dir) if f.endswith('.nii')]
rsn_files.sort()

In [5]:
# # # Create A 4D volume with all 506 maps of the model
# this may take a few min
terms_maps_dir = f'{project_dir}/data/dataset'
terms_maps_files = [os.path.join(terms_maps_dir, f) for f in os.listdir(terms_maps_dir) if f.endswith('.nii.gz')]
terms_maps_files.sort()

out_dir = f'{terms_maps_dir}/concatenated'
os.makedirs(out_dir, exist_ok=True)

out_name = out_dir + '/dataset_concatenated.nii.gz'
if os.path.isfile(out_name) == False:
    image.concat_imgs(terms_maps_files).to_filename(out_name)
concat_ds = image.load_img(out_name)

terms_maps_files_all = terms_maps_files

In [6]:
# Create & retrieve parcellated 506 meta analytic maps
# should also take a few minutes, but needs to be done only once

n_terms = concat_ds.shape[3]

parcellated_dataset_dir = f'{project_dir}/data/dataset/parcellated/{atlas_str}'
os.makedirs(parcellated_dataset_dir, exist_ok=True)

parcellated_dataset_file = os.path.join(parcellated_dataset_dir, f'neurosynthterms_parcellations_{atlas_str}.csv')

parcellated = []

if os.path.isfile(parcellated_dataset_file) == False:
    for t in range(n_terms):
        temp_img = image.index_img(concat_ds, t)
        parcellated.append(fcu.parcellate(temp_img, atlas=parcellation_atlas_file))

    parcellated = np.array(parcellated).squeeze()
    terms = neurosynth_terms.values
    # print(parcellated.shape)
    parcels = [i for i in range(1, parcellated.shape[1] +1)]
    df = pd.DataFrame(parcellated, index=terms, columns = parcels)
    df.to_csv(parcellated_dataset_file)

dataset_parcellated = pd.read_csv(parcellated_dataset_file, sep = ',', index_col = 0)



In [7]:
# for each rsn, compute the spatial correlation with the 506 meta analytic maps (parcellated), sort
atlas_img = '/homes_unix/agillig/Atlases/RSN_N41_zNpair_clean1.nii'
spatial_correlation = {}
parcellation_dir = analysis_dir + '/parcellations'
os.makedirs(parcellation_dir, exist_ok=True)


rsn_str = pd.DataFrame(rsn_files).iloc[:,0].str.split('/').str[-1].str[-6:-4]

for i, rsn in enumerate(rsn_str):
    parcellation_file = parcellation_dir + f'/rsn-{rsn}/rsn-{rsn}_unique_parcellated.csv'
    os.makedirs(os.path.dirname(parcellation_file), exist_ok=True)

    index = int(rsn) - 1
    # tmp_data = image.index_img(atlas_img, index)
    # affine = tmp_img.affine
    if os.path.isfile(parcellation_file) == False:
        tmp_img = image.load_img(rsn_files[index])
        tmp_parcellatd = fcu.parcellate(tmp_img, atlas=parcellation_atlas_file)
        np.savetxt(parcellation_file, tmp_parcellatd, delimiter = ',')
    data = np.loadtxt(parcellation_file, delimiter = ',')


    corr_temp = [np.corrcoef(data, dataset_parcellated.iloc[j,:])[1,0] for j in range(n_terms)]

    # corr_temp = []
    # for j, (term, file) in enumerate(zip(BCSterms, BCS_maps_files)): 
    #     img_comparison = image.index_img(concat_ds, j)
    #     data_comparison = img_comparison.get_fdata().copy()
    #     corr_temp.append(np.corrcoef(data.flatten(), data_comparison.flatten())[1,0])
    spatial_correlation[rsn] = corr_temp

# Non parametric statistics

## Uncorrected

In [28]:
# null_projections = {}
# use brainsmash to generate surrogate data that preserves spatial autocorrelation (Burt, 2020)
# https://github.com/murraylab/brainsmash: cf null_parcellations.py

# once computed, for each rsn, compute the correlation of the null parcellations with the 506 meta analytic maps (parcellated)

n_perm = 100 # number of permutations per batch
n_batches = 1 # number of batches. can be useful to split the work in a cluster environment
total_n_perm = int(n_perm * n_batches)

null_dir = analysis_dir + '/null_correlations'
def compute_correlation_null(rsn, n_perm = 1000, n_batches = 10, overwrite = False):
    print(f'processing rsn {rsn}')

    null_dir = analysis_dir + '/null_correlations'
    save_dir = null_dir + f'/rsn-{rsn}'
    os.makedirs(save_dir, exist_ok=True)
    save_file = os.path.join(save_dir, f'rsn-{rsn}_null_correlations.csv')    

    if os.path.isfile(save_file) and overwrite == False:
        print('file already exists; no overwriting')
        return
    
    data = []
    for batch in range(1,n_batches+1):
        null_prcl_dir = analysis_dir + '/null_parcellations'
        null_parcellations_file = null_prcl_dir + f'/rsn-{rsn}/rsn-{rsn}_null_parcellations_batch-{batch:02d}_of_{n_batches}.csv'
        temp_data = np.loadtxt(null_parcellations_file, delimiter = ' ')
        data.append(temp_data)
    data = np.concatenate(data, axis = 0)
    # print(f'shape of data: {data.shape}')

    index = int(rsn) - 1

    # Get the target series
    corr_temp = []
    for j in range(n_terms):
        
        term_single = dataset_parcellated.iloc[j,:].values
        # Compute the Pearson correlation coefficient for each column
        corr_temp.append([np.corrcoef(data.T[:, i], term_single)[0,1] for i in range(n_perm * n_batches)])

        # corr_temp = []
        # for j, (term, file) in enumerate(zip(BCSterms, BCS_maps_files)): 
        #     img_comparison = image.index_img(concat_ds, j)
        #     data_comparison = img_comparison.get_fdata().copy()
        #     corr_temp.append(np.corrcoef(data.flatten(), data_comparison.flatten())[1,0])
    corr_temp = np.array(corr_temp)
    # spatial_correlation_null[rsn] = corr_temp
    # save to file
    print('saving file')
    np.savetxt(save_file, corr_temp) 
    return corr_temp

In [9]:
# # save null correlations for each rsn
# from multiprocessing import Pool
# spatial_correlation_null = {}
# with Pool() as pool:
#     pool.map(compute_correlation_null, rsn_str)

In [25]:
from null_parcellations import generate_null
import importlib
import null_parcellations
importlib.reload(null_parcellations)
null_parcellations.generate_null(1, project_dir, n_perm=10, n_batches=n_batches)

processing rsn 01
batch 1
generating 10 surrogates
elapsed time: 2.85 s
saving surrogate maps to /homes_unix/agillig/github_repos/ginna/analysis/mean_RSNs_aicha-aal3/null_parcellations/rsn-01/rsn-01_null_parcellations_batch-01_of_1.csv


In [29]:
res = compute_correlation_null('01', n_perm=10, n_batches=1, overwrite=True)

processing rsn 01
saving file


# stopped here

In [18]:
# load null correlations

null_correlations = []
null_correlations_dir = analysis_dir + '/null_correlations'
null_prcl_dir = analysis_dir + 'null_parcellations'

# from brainsmash.mapgen.base import Base
n_perm = 10000
n_batches = 10


# os.makedirs(null_dir, exist_ok=True)
print(f'loading null correlations...')
for ind, rsn in enumerate(rsn_str):
    
    save_dir = null_correlations_dir + f'/rsn-{rsn}'

    save_file = os.path.join(save_dir, os.listdir(save_dir)[0])
        
    null_correlations.append(np.loadtxt(save_file))
                             
null_correlations = np.asarray(null_correlations)

loading null correlations...


In [19]:
# 1. compute p-value for each term / RSN association
# compute p-value for each term / RSN association
n_terms = len(neurosynth_terms)
# p_val_list = []

def compute_all_pval_uncor(ind):
    rsn = rsn_str[ind]

    null_distribution_dir = null_dir + f'/rsn-{rsn}'
    null_distribution_file =  os.path.join(null_distribution_dir, f'rsn-{rsn}_null_correlations.csv')  

    # print(f'processing rsn {rsn}')
    parcellation_file = parcellation_dir + f'/rsn-{rsn}/rsn-{rsn}_unique_parcellated.csv'
    data_parcellated = np.loadtxt(parcellation_file, delimiter = ',')
    temp_pvals = []
    observed_corr_list = []
    null_distribution = np.loadtxt(null_distribution_file, delimiter = ' ')
    tmp_ind_obs = []
    for t in range(n_terms): 
        # print(f'BCS term: {BCSterms[t]}')
        
        observed_corr = spatial_correlation[rsn][t]
        # print(f'observed correlation: {observed_corr}')
        observed_corr_list.append(observed_corr)
        null_corr= []

        # without correction for multiple comparisons
        null_distribution_term = null_distribution[t,:]
        distr_sorted = np.sort(null_distribution_term)

        n_val = len(distr_sorted) + 1
        ind_obs = n_val - np.searchsorted(distr_sorted, observed_corr) #+1 # +1 because python indices start at 0
        # print(ind_obs)
        tmp_ind_obs.append(ind_obs)
        p_val = ind_obs / n_val
        temp_pvals.append(p_val)
    temp_pvals = np.asarray(temp_pvals)
    tmp_ind_obs = np.asarray(tmp_ind_obs)
    observed_corr_list = np.asarray(observed_corr_list)
    # p_val_list.append(temp_pvals)
    output = {'pvals': temp_pvals, 'indice': tmp_ind_obs, 'observed_correlation': observed_corr_list}
    return output

In [20]:
from multiprocessing import Pool
with Pool() as pool:
    results_uncor = pool.map(compute_all_pval_uncor, range(len(rsn_str)))

In [21]:
pvals_all_uncor = np.asarray([results_uncor[i]['pvals'] for i in range(len(rsn_str))])
observed_correlations_all_uncor = np.asarray([results_uncor[i]['observed_correlation'] for i in range(len(rsn_str))])

p_arr = np.hstack(pvals_all_uncor)
correlation_arr = np.hstack(observed_correlations_all_uncor)

## Correction for multiple comparisons
repeat the same but with correction for multiple comparisons

In [22]:
# maximal statistics correction accross the terms (not rsn)
def compute_null_distr_corr(rsn):
    _n_perm = 10000

    # load null distribution (n_parcels x n_perm)
    null_distribution_dir = null_dir + f'/rsn-{rsn}'
    null_distribution_file =  os.path.join(null_distribution_dir, f'rsn-{rsn}_null_correlations.csv')  
    # print(f'processing rsn {rsn}')
    null_distribution = np.loadtxt(null_distribution_file, delimiter = ' ')

    corrected_null_distribution = np.max(null_distribution, axis=0)
    

    save_dir = analysis_dir + f'/null_correlations_maxcorc/rsn-{rsn}'
    os.makedirs(save_dir, exist_ok=True)
    savefile = save_dir + f'/rsn-{rsn}_null_correlations_maxcorc.csv'

    np.savetxt(savefile, corrected_null_distribution, delimiter=',')

In [23]:
from multiprocessing import Pool
with Pool() as pool:
    pool.map(compute_null_distr_corr, rsn_str)

In [24]:
# correction function accross terms
def compute_all_pval_cor(ind):
    rsn = rsn_str[ind]

    null_distribution_dir = analysis_dir + f'/null_correlations_maxcorc/rsn-{rsn}'
    null_distribution_file =  os.path.join(null_distribution_dir, f'rsn-{rsn}_null_correlations_maxcorc.csv')  

    # print(f'processing rsn {rsn}')
    temp_pvals = []
    observed_corr_list_corc = []
    null_distribution = np.loadtxt(null_distribution_file, delimiter = ' ')
    distr_sorted = np.sort(null_distribution)

    tmp_ind_obs = []
    for t in range(n_terms): 
        # print(f'BCS term: {BCSterms[t]}')
        
        observed_corr = spatial_correlation[rsn][t]
        # print(f'observed correlation: {observed_corr}')
        observed_corr_list_corc.append(observed_corr)
        null_corr= []

        # without correction for multiple comparisons
        # null_distribution_term = null_distribution
        # distr_sorted = np.sort(null_distribution_term)

        n_val = len(distr_sorted) + 1
        ind_obs = n_val - np.searchsorted(distr_sorted, observed_corr) #+1 # +1 because python indices start at 0
        # print(ind_obs)
        tmp_ind_obs.append(ind_obs)
        p_val = ind_obs / n_val
        temp_pvals.append(p_val)
    temp_pvals = np.asarray(temp_pvals)
    tmp_ind_obs = np.asarray(tmp_ind_obs)
    observed_corr_list_corc = np.asarray(observed_corr_list_corc)
    # p_val_list.append(temp_pvals)
    output = {'pvals': temp_pvals, 'indice': tmp_ind_obs, 'observed_correlation': observed_corr_list_corc}
    return output

In [25]:
from multiprocessing import Pool
with Pool() as pool:
    results_cor = pool.map(compute_all_pval_cor, range(len(rsn_str)))

In [26]:
import json

savefile = project_dir + '/Results/files/aicha/rsn_p_maxcorc_termwise.csv'
os.makedirs(os.path.dirname(savefile), exist_ok=True)
save_correlation = {}
save_cor = {}
save_p_uncor = {}
for i, rsn in enumerate(rsn_str):
    save_cor[rsn] = results_cor[i]['pvals'].tolist()
    save_p_uncor[rsn] = results_uncor[i]['pvals'].tolist()
    save_correlation[rsn] = results_cor[i]['observed_correlation'].tolist()
# save p values
    # corrected
with open(savefile, 'w') as f:
    json.dump(save_cor, f)
    # uncorrected
savefile = project_dir + '/Results/files/aicha/rsn_p_uncor.csv'
with open(savefile, 'w') as f:
    json.dump(save_p_uncor, f)

# save correlations
savefile = project_dir + '/Results/files/aicha/rsn_Pearsonr.csv'
with open(savefile, 'w') as f:
    json.dump(save_correlation, f)

In [27]:
pvals_all_cor = np.asarray([results_cor[i]['pvals'] for i in range(len(rsn_str))])
observed_correlations_all_cor = np.asarray([results_cor[i]['observed_correlation'] for i in range(len(rsn_str))])

pcor_arr = np.hstack(pvals_all_cor)
correlationcor_arr = np.hstack(observed_correlations_all_cor)

# Assume groups is a list or array of the same length as correlation_arr and p_arr
# that defines the group for each point
groups = np.hstack([[rsn for i in range(pvals_all_cor[0].shape[0])] for rsn in rsn_str])

In [28]:
selected_terms = {}
pvals = {}
selected_terms_list = []
pvals_list = []
for i, rsn in enumerate(rsn_str):
    selected_terms[rsn] = neurosynth_terms[results_cor[i]['pvals'] < 0.05].tolist()
    pvals[rsn] = results_cor[i]['pvals'][results_cor[i]['pvals'] < 0.05]
    selected_terms_list.append(neurosynth_terms[results_cor[i]['pvals'] < 0.05].tolist())
    pvals_list.append(results_cor[i]['pvals'][results_cor[i]['pvals'] < 0.05])

In [29]:
from itertools import chain
rsn_row_id = [np.repeat(rstr, 1).tolist() for i, rstr in enumerate(rsn_str)]
rsn_row_id = np.asarray(list(chain.from_iterable(rsn_row_id)))
cluster_row_id = [f'{1:02d}' for i in range(n_rsn)] 
cluster_row_id = np.asarray(list(chain.from_iterable(cluster_row_id)))
indexing_array = [rsn_row_id, cluster_row_id]
rsn_id_group = rsn_row_id

In [30]:
list_terms_sel = []
str_terms_sel_p = []
str_p_sel = []
list_dist_sel = []

for i, rsn in enumerate(rsn_str):

        dst = observed_correlations_all_cor[i]
        pval = pvals_all_cor[i] 

        sel_terms_p = neurosynth_terms[pval < 0.05].tolist()
        sel_p = pval[pval < 0.05]

        def format_decimal(val):
                # s = str(val)
                s = "{:.15f}".format(val)  # Convert the number to a string with 15 decimal places; avoids scientific notation bypass of method
                if '.' in s:
                        pre, post = s.split('.')
                        first_non_zero = next((i for i, c in enumerate(post) if c != '0'), len(post))
                        return f'{{:.{first_non_zero + 1}f}}'.format(val)
                else:
                        return s

        sel_p = [format_decimal(float(i)) for i in sel_p]

        # sel_p = [f'{float(i):.1f}'.lstrip('0') for i in sel_p]
        sel_terms_p = [f'{tstr} ({pstr})' for tstr, pstr in zip(sel_terms_p, sel_p)]

        # list_terms_sel.append(BCSterms[np.argsort(d)][:k])
        tstr = ', '.join(sel_terms_p)
        pstr = ', '.join(sel_p)
        # list_dist_sel.append(d[np.argsort(d)][:k])
        str_terms_sel_p.append(np.asarray(tstr, dtype='str'))
        str_p_sel.append(np.asarray(pstr, dtype='str'))